In [ ]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [ ]:
data = [
    ("1000", 0, 100),
    ("1001", 1, 100),
    ("1002", 2, 101),
    ("1003", 3, 102),
    ("1004", 4, 103),
    ("1005", 5, 100),
    ("1005", 6, 100)
]

In [ ]:
df_left = spark.createDataFrame(data, schema=['id', 'left', 'right'])
df_left.show()

+----+----+-----+
|  id|left|right|
+----+----+-----+
|1000|   0|  100|
|1001|   1|  100|
|1002|   2|  101|
|1003|   3|  102|
|1004|   4|  103|
|1005|   5|  100|
|1005|   6|  100|
+----+----+-----+



In [ ]:
data1 = [
    ("1000", "john", 100),
    ("1001", "marcel", 100),
    ("1002", "yves", 101),
    ("1003", "rom", 102),
    ("1004", "mour", 103),
    ("1005", "mike", 100),
    ("1005", "mor", 100)
]

In [ ]:
df_right = spark.createDataFrame(data1, schema=['id', 'left1', 'right1'])
df_right.show()

+----+------+------+
|  id| left1|right1|
+----+------+------+
|1000|  john|   100|
|1001|marcel|   100|
|1002|  yves|   101|
|1003|   rom|   102|
|1004|  mour|   103|
|1005|  mike|   100|
|1005|   mor|   100|
+----+------+------+



In [ ]:
df1 = df_left.join(df_right, on = "id", how = "full")
df1.show()

+----+----+-----+------+------+
|  id|left|right| left1|right1|
+----+----+-----+------+------+
|1004|   4|  103|  mour|   103|
|1000|   0|  100|  john|   100|
|1005|   5|  100|  mike|   100|
|1005|   5|  100|   mor|   100|
|1005|   6|  100|  mike|   100|
|1005|   6|  100|   mor|   100|
|1003|   3|  102|   rom|   102|
|1002|   2|  101|  yves|   101|
|1001|   1|  100|marcel|   100|
+----+----+-----+------+------+



In [ ]:
df1.groupBy("left1")\
.agg({"right" : "sum"})\
.orderBy("left1").show()

+------+----------+
| left1|sum(right)|
+------+----------+
|  john|       100|
|marcel|       100|
|  mike|       200|
|   mor|       200|
|  mour|       103|
|   rom|       102|
|  yves|       101|
+------+----------+



In [ ]:
df = spark.createDataFrame(
 [("sales",1,7000),("sales",2,6000),("sales",9,5000),("sales",10,6000),("hr",3,2000),("hr",4,6000),("hr",7,3000),("it",5,5000),("it",6,11000),("it",8,19000)],["dept_id","emp_id","salary"])
df.show()

+-------+------+------+
|dept_id|emp_id|salary|
+-------+------+------+
|  sales|     1|  7000|
|  sales|     2|  6000|
|  sales|     9|  5000|
|  sales|    10|  6000|
|     hr|     3|  2000|
|     hr|     4|  6000|
|     hr|     7|  3000|
|     it|     5|  5000|
|     it|     6| 11000|
|     it|     8| 19000|
+-------+------+------+



In [67]:
from pyspark.sql import functions as F
from pyspark.sql import Window

In [68]:
window = Window.partitionBy("dept_id")
window = window.orderBy("dept_id")
window = window.rowsBetween(-2, 0)

In [69]:
window = Window.partitionBy("dept_id")
window = window.orderBy("dept_id")
window = window.rowsBetween(-2,0)

In [ ]:
df.withColumn("sum", F.sum("salary").over(window)).orderBy("dept_id").show()

+-------+------+------+-----+
|dept_id|emp_id|salary|  sum|
+-------+------+------+-----+
|     hr|     7|  3000|11000|
|     hr|     4|  6000| 8000|
|     hr|     3|  2000| 2000|
|     it|     5|  5000| 5000|
|     it|     8| 19000|35000|
|     it|     6| 11000|16000|
|  sales|     1|  7000| 7000|
|  sales|     2|  6000|13000|
|  sales|     9|  5000|18000|
|  sales|    10|  6000|17000|
+-------+------+------+-----+



how to calcule the lag

In [70]:
window = Window.partitionBy("dept_id").orderBy(F.col("Salary").desc())

In [71]:
df.withColumn("lag", F.lag("Salary", 1).over(window)).show()

+-------+------+------+-----+
|dept_id|emp_id|salary|  lag|
+-------+------+------+-----+
|  sales|     1|  7000| null|
|  sales|     2|  6000| 7000|
|  sales|    10|  6000| 6000|
|  sales|     9|  5000| 6000|
|     hr|     4|  6000| null|
|     hr|     7|  3000| 6000|
|     hr|     3|  2000| 3000|
|     it|     8| 19000| null|
|     it|     6| 11000|19000|
|     it|     5|  5000|11000|
+-------+------+------+-----+



In [72]:
data = [ (0, "This is the first document"),   (1, "This document is the second document"),
    (2, "And this is the third one"),   (3, "Is this the first document?"),
    (4, "The last document is the fifth one") ]
df = spark.createDataFrame(data, ["id", "text"])
df.show(truncate=False)

+---+------------------------------------+
|id |text                                |
+---+------------------------------------+
|0  |This is the first document          |
|1  |This document is the second document|
|2  |And this is the third one           |
|3  |Is this the first document?         |
|4  |The last document is the fifth one  |
+---+------------------------------------+



In [74]:
from pyspark.ml.feature import Tokenizer, IDF, HashingTF

In [81]:
tokenizer = Tokenizer(inputCol = "text", outputCol = "words")

In [82]:
words = tokenizer.transform(df)

In [85]:
words.show(truncate=False)

+---+------------------------------------+-------------------------------------------+
|id |text                                |words                                      |
+---+------------------------------------+-------------------------------------------+
|0  |This is the first document          |[this, is, the, first, document]           |
|1  |This document is the second document|[this, document, is, the, second, document]|
|2  |And this is the third one           |[and, this, is, the, third, one]           |
|3  |Is this the first document?         |[is, this, the, first, document?]          |
|4  |The last document is the fifth one  |[the, last, document, is, the, fifth, one] |
+---+------------------------------------+-------------------------------------------+



In [95]:
hasingTF = HashingTF(inputCol = "words", outputCol = "rawFeatures", numFeatures=15)

In [96]:
featureTF = hasingTF.transform(words)

In [97]:
featureTF.show(truncate=False)

+---+------------------------------------+-------------------------------------------+--------------------------------------------+
|id |text                                |words                                      |rawFeatures                                 |
+---+------------------------------------+-------------------------------------------+--------------------------------------------+
|0  |This is the first document          |[this, is, the, first, document]           |(15,[2,3,4,10,13],[1.0,1.0,1.0,1.0,1.0])    |
|1  |This document is the second document|[this, document, is, the, second, document]|(15,[2,3,4,6,10],[1.0,1.0,1.0,1.0,2.0])     |
|2  |And this is the third one           |[and, this, is, the, third, one]           |(15,[0,1,2,3,4,6],[1.0,1.0,1.0,1.0,1.0,1.0])|
|3  |Is this the first document?         |[is, this, the, first, document?]          |(15,[1,2,3,4,13],[1.0,1.0,1.0,1.0,1.0])     |
|4  |The last document is the fifth one  |[the, last, document, is, the, fif

In [100]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [102]:
idfF = idf.fit(featureTF)

In [107]:
idfF.transform(featureTF).show(truncate=False)

+---+------------------------------------+-------------------------------------------+--------------------------------------------+--------------------------------------------------------------------------------------------------------+
|id |text                                |words                                      |rawFeatures                                 |features                                                                                                |
+---+------------------------------------+-------------------------------------------+--------------------------------------------+--------------------------------------------------------------------------------------------------------+
|0  |This is the first document          |[this, is, the, first, document]           |(15,[2,3,4,10,13],[1.0,1.0,1.0,1.0,1.0])    |(15,[2,3,4,10,13],[0.0,0.1823215567939546,0.0,0.4054651081081644,0.6931471805599453])                   |
|1  |This document is the second document|[this, doc

In [108]:
from pyspark.ml.feature import Word2Vec, Tokenizer

In [109]:
data = [     (0, ["apple", "banana", "orange", "grape"]),     (1, ["apple", "banana", "cherry", "pear"]),
    (2, ["banana", "cherry", "grape", "kiwi"]),     (3, ["apple", "pear", "kiwi", "orange"]),
   (4, ["cherry", "grape", "kiwi", "orange"]) ]

In [110]:
data

[(0, ['apple', 'banana', 'orange', 'grape']),
 (1, ['apple', 'banana', 'cherry', 'pear']),
 (2, ['banana', 'cherry', 'grape', 'kiwi']),
 (3, ['apple', 'pear', 'kiwi', 'orange']),
 (4, ['cherry', 'grape', 'kiwi', 'orange'])]

In [111]:
df = spark.createDataFrame(data, ["id", "words"])

In [113]:
df.show( truncate= False)

+---+------------------------------+
|id |words                         |
+---+------------------------------+
|0  |[apple, banana, orange, grape]|
|1  |[apple, banana, cherry, pear] |
|2  |[banana, cherry, grape, kiwi] |
|3  |[apple, pear, kiwi, orange]   |
|4  |[cherry, grape, kiwi, orange] |
+---+------------------------------+



In [121]:
word2vec = Word2Vec(inputCol="words", outputCol="features", vectorSize=1, minCount=0)

In [122]:
model = word2vec.fit(df)

In [123]:
model.transform(df).show(truncate=False)

+---+------------------------------+-----------------------+
|id |words                         |features               |
+---+------------------------------+-----------------------+
|0  |[apple, banana, orange, grape]|[0.12237552553415298]  |
|1  |[apple, banana, cherry, pear] |[0.061410319060087204] |
|2  |[banana, cherry, grape, kiwi] |[0.12669403105974197]  |
|3  |[apple, pear, kiwi, orange]   |[0.0568082295358181]   |
|4  |[cherry, grape, kiwi, orange] |[-0.035940058529376984]|
+---+------------------------------+-----------------------+



In [124]:
from pyspark.ml.feature import OneHotEncoder
data = [(0.0, 1.0), (1.0, 0.0), (2.0, 1.0)]
columns = ["input1", "input2"]
df = spark.createDataFrame(data, columns)
df.show()

+------+------+
|input1|input2|
+------+------+
|   0.0|   1.0|
|   1.0|   0.0|
|   2.0|   1.0|
+------+------+



In [125]:
encoder = OneHotEncoder(inputCols=["input1", "input2"], outputCols=["output1", "output2"])
encoded_df = encoder.fit(df)
encoded_df = encoded_df.transform(df)
encoded_df.select("output1","output2").show(truncate=False)

+-------------+-------------+
|output1      |output2      |
+-------------+-------------+
|(2,[0],[1.0])|(1,[],[])    |
|(2,[1],[1.0])|(1,[0],[1.0])|
|(2,[],[])    |(1,[],[])    |
+-------------+-------------+

